In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [18]:
edges = pd.read_csv('/home/manoj/tuhh/data_science_3rd_sem/deep_learning_for_social_analytics/project/deep-learning-for-social-analytics-project/notebooks/edges_cleaned.csv')
nodes = pd.read_csv('/home/manoj/tuhh/data_science_3rd_sem/deep_learning_for_social_analytics/project/deep-learning-for-social-analytics-project/notebooks/nodes_cleaned.csv')

In [19]:
nodes.describe()

,followers,popularity,alternative Indie,classical_orchestral,electronic,folk world,hazz,hip_hop,latin,metal,pop,randb_Soul,reggae_dancehall,rock,soundtrack,unknown
count,1.438480e+05,1.438480e+05,143848.000000,143848.000000,143848.000000,143848.000000,143848.000000,143848.000000,143848.000000,143848.000000,143848.000000,143848.000000,143848.000000,143848.000000,143848.000000,143848.000000
mean,3.437919e-17,-6.796805e-17,0.018825,0.008057,0.037275,0.009169,0.006695,0.079508,0.005923,0.002280,0.109484,0.023601,0.006264,0.026438,0.002412,0.162880
std,1.000003e+00,1.000003e+00,0.135909,0.089399,0.189437,0.095317,0.081546,0.270530,0.076733,0.047697,0.312246,0.151804,0.078895,0.160433,0.049056,0.369258
min,-9.142721e-02,-1.166466e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-9.140032e-02,-9.488909e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,-9.102273e-02,-1.329843e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,-8.465031e-02,7.373161e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.055881e+02,4.272911e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [20]:
nodes.head()

,spotify_id,name,followers,popularity,alternative Indie,classical_orchestral,electronic,folk world,hazz,hip_hop,latin,metal,pop,randb_Soul,reggae_dancehall,rock,soundtrack,unknown
0,48WvrUGoijadXXCsGocwM4,Byklubben,-0.089629,0.138985,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,4lDiJcOJ2GLCK6p9q5BgfK,Kontra K,1.977204,2.749886,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,652XIvIBNGg3C0KIGEJWit,Maxim,-0.055638,0.791710,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3dXC1YPbnQPsfHPVkm1ipj,Christopher Martin,0.166400,1.662010,0,0,0,0,0,0,0,0,0,0,1,1,0,1
4,74terC9ol9zMo8rfzhSOiG,Jakob Hellman,-0.069503,0.954891,1,0,0,0,0,0,0,0,1,0,0,0,0,0


In [21]:
edges.head()

,id_0,id_1,artist_0,artist_1
0,76M2Ekj8bG8W7X2nbx2CpF,7sfl4Xt5KmfyDs2T3SVSMK,NGHTMRE,Lil Jon
1,0hk4xVujcyOr6USD95wcWb,7Do8se3ZoaVqUt3woqqSrD,Sick Luke,MACHETE
2,38jpuy3yt3QIxQ8Fn1HTeJ,4csQIMQm6vI2A2SCVDuM2z,Mat.Joe,Hayden James
3,6PvcxssrQ0QaJVaBWHD07l,6UCQYrcJ6wab6gnQ89OJFh,Lp2loose,Headie One
4,2R1QrQqWuw3IjoP5dXRFjt,4mk1ScvOUkuQzzCZpT6bc0,Marlon Roudette,Stadic


In [22]:
# Merge edges with node attributes for artist_0
merged_0 = pd.merge(edges, nodes, left_on='id_0', right_on='spotify_id', suffixes=('_artist_0', '_artist_1'))

# Merge the resulting DataFrame with node attributes for artist_1
full_merged = pd.merge(merged_0, nodes, left_on='id_1', right_on='spotify_id', suffixes=('_artist_0', '_artist_1'))

In [23]:
def compute_pair_features(row):
    # Absolute difference in popularity
    row['popularity_diff'] = abs(row['popularity_artist_0'] - row['popularity_artist_1'])

    # Absolute difference in followers
    row['follower_diff'] = abs(row['followers_artist_0'] - row['followers_artist_1'])

    # Genre similarity (dot product or count of matching genres)
    genre_columns = ['alternative Indie', 'classical_orchestral', 'electronic', 'folk world', 'hazz', 
                     'hip_hop', 'latin', 'metal', 'pop', 'randb_Soul', 'reggae_dancehall', 'rock', 
                     'soundtrack', 'unknown']
    
    artist_0_genres = row[[f'{col}_artist_0' for col in genre_columns]].values
    artist_1_genres = row[[f'{col}_artist_1' for col in genre_columns]].values
    # Dot product for similarity score
    row['genre_similarity'] = sum(artist_0_genres * artist_1_genres)
    
    return row

# Apply feature engineering function
full_merged = full_merged.apply(compute_pair_features, axis=1)

In [24]:
# Define the features and label
features = ['popularity_diff', 'follower_diff', 'genre_similarity']
X = full_merged[features]
# Label is 1 for all true collaborations in your edges DataFrame
y = pd.Series([1] * len(X))  


In [25]:
print(X)
print("*"*50)
print(y)

        popularity_diff  follower_diff  genre_similarity
0              0.543938       0.931226                 2
1              0.271969       1.128125                 3
2              1.468632       1.360135                 2
3              0.979088       1.241481                 0
4              1.903782       1.283599                 1
...                 ...            ...               ...
267401         1.196663       0.012904                 0
267402         1.414238       1.509919                 0
267403         2.447720       0.041221                 0
267404         1.414238       0.031120                 0
267405         0.000000       0.000771                 0

[267406 rows x 3 columns]
**************************************************
0         1
1         1
2         1
3         1
4         1
         ..
267401    1
267402    1
267403    1
267404    1
267405    1
Length: 267406, dtype: int64


#### Rest Varad and Richard can have a look.
Data_pre_processing: nodes.
1. Genres are one hot encoding.
2. followers and rank did the standard_scaler as it's ordinal data.
3. Kept the name and spotify id as it is apart from cleaning null values and removing duplicates and remove non english names.

edges:
1. Cleaned null values and removing duplicates and remove non english names.
2. Properly mapped the spotify id with the names
3. id_0 is mapped to artist_0 and id_1 is mapped to artist_1